in this notebook we will classify a clients sentiments reviews based on some training dataset

In [ ]:
import numpy as np 
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten , Embedding

In [128]:
reviews = [
    'Nice service',
    'Delisious food', 
    "Amazing place to eat",
    "Not satisfied",
    "poor service",
    "Just love it!",
    "Needs improvements",
    "Bad food quality",
    "Never come back here!",
    "too good"
]

labels= np.array([1,1,1,0,0,1,0,0,0,1])

the one_hot fonction encodes a text into a list of word indexes (integers) of size 'n' that we specify as second argument of the fonction.

it gives us unique integers (between 0 and n where n is the second argument of the fonction) for each word in the input_text. 

In [45]:
one_hot("Hey There, How are you !",10)

[7, 3, 8, 9, 4]

In [117]:
# now we will encode all the reviews into integers 
vocab_size =50
encoded_reviews = [one_hot(words,vocab_size) for words in reviews]
print(encoded_reviews)

[[11, 24], [42, 44], [1, 41, 21, 45], [7, 40], [27, 24], [36, 1, 25], [39, 8], [2, 44, 10], [6, 35, 29, 8], [4, 26]]


we need to padd every sequence of words into the len of the largest sequence in the vocab

In [118]:
max_len = max([len(w.split()) for w in reviews])
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_len, padding='post')
print(padded_reviews)

[[11 24  0  0]
 [42 44  0  0]
 [ 1 41 21 45]
 [ 7 40  0  0]
 [27 24  0  0]
 [36  1 25  0]
 [39  8  0  0]
 [ 2 44 10  0]
 [ 6 35 29  8]
 [ 4 26  0  0]]


now all the sequences are padded, and they all have the same size 

the next step is to create the model and for this we must define the embeded vector size, which is the size of the ouput vectors of the Embedding Layer

In [141]:
embaded_vector_size = 5
model = Sequential()
model.add(Embedding(input_dim = vocab_size,output_dim= 8, input_length = max_len,name = 'embedding'))
model.add(Flatten())
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer = 'adam',loss="binary_crossentropy",metrics =["accuracy"])
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 8)              400       
_________________________________________________________________
flatten_20 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________


In [142]:
model.fit(padded_reviews, labels, epochs=50, verbose=0)


In [143]:
loss, acc = model.evaluate(padded_reviews,labels,verbose=0)
print(f"\n\nThe loss : {loss}, accuracy : {acc}")



The loss : 0.5743067860603333, accuracy : 1.0


In [156]:
weights = model.get_layer("embedding").get_weights()[0]

In [166]:
weights[11]

array([ 0.02778094, -0.02430862, -0.03690113, -0.07403668, -0.00286919,
       -0.0463014 , -0.07022117, -0.05354059], dtype=float32)

In [167]:
weights[42]

array([ 0.0722071 , -0.09330942, -0.03600999, -0.07076837, -0.06134059,
       -0.01724471, -0.01825108, -0.06911809], dtype=float32)

here we see that the 2 words nice and delicious have almost the same vectors and that because they are similaire.

so for a big dataset can perform the weights, so that all the similaire values have the same vectors 